<a href="https://colab.research.google.com/github/ambideXtrous9/Indian-NewsQA/blob/main/BERTScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --quiet bertserini
!pip install --quiet bert-score
!pip install --quiet transformers==3.5
!pip install --quiet torchmetrics

     |████████████████████████████████| 6.6 MB 7.4 MB/s 
     |████████████████████████████████| 753.2 MB 14 kB/s 
     |████████████████████████████████| 5.5 MB 59.1 MB/s 
     |████████████████████████████████| 276 kB 70.5 MB/s 
     |████████████████████████████████| 125 kB 66.3 MB/s 
     |████████████████████████████████| 60.4 MB 1.2 MB/s 
     |████████████████████████████████| 1.3 MB 57.5 MB/s 
     |████████████████████████████████| 7.6 MB 54.5 MB/s 
     |████████████████████████████████| 163 kB 74.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.5.1 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.5.1 which is incompatible.
fastai 2.7.9 requires torch<1.14,>=1.7, but you have torch 1.5.1 which is incompatible.
fastai 2.7.9 requires torchvisi

In [3]:
import pandas as pd

In [4]:
qapath = '/content/drive/MyDrive/MTP CODE/NewsQA.feather'

In [5]:
df = pd.read_feather(qapath)
df

,question,answer,paragraph
0,Who is the managing director of Synergee Capital?,Vikram Dalal,"""Investors can use a combination of governmen..."
1,What is the yield of 30- and 40-year governmen...,7%,"""Investors can use a combination of governmen..."
2,What is the name of the ETF 2027 that a conser...,SDL,"According to financial planners, an example o..."
3,When would a conservative fixed income investo...,2027,"According to financial planners, an example o..."
4,What year would a conservative fixed income in...,2040,"According to financial planners, an example o..."
...,...,...,...
481753,When does Uncle Sam reopen for fully vaccinate...,November 8,NEW DELHI: This could be the last expansion of...
481754,When will there be three more weekly flights b...,from second week of November,It currently has 23 weekly flights to America....
481755,What type of 777s would have helped AI have mo...,Boeing,It currently has 23 weekly flights to America....
481756,What was the first wave of AI nonstops?,second,"Before the second wave this summer, AI had abo..."


In [6]:
(df['answer'].values == '').sum()

0

In [7]:
df.isnull().sum()

question     0
answer       0
paragraph    0
dtype: int64

In [8]:
from bertserini.reader.base import Question, Context
from bertserini.reader.bert_reader import BERT
from bertserini.utils.utils_new import get_best_answer

In [9]:
model_name = "rsvp-ai/bertserini-bert-base-squad"
tokenizer_name = "rsvp-ai/bertserini-bert-base-squad"
bert_reader = BERT(model_name, tokenizer_name)

Downloading:   0%|          | 0.00/477 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [10]:
!pip install --quiet bert_score

In [11]:
from bert_score import BERTScorer

In [12]:
scorer = BERTScorer(lang="en", rescale_with_baseline=True)

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

In [13]:
df1 = df.sample(frac =.03)

In [14]:
df1.shape[0]

14453

In [15]:
df1.head()

,question,answer,paragraph
85062,What is the name of Gujarat Co-operative Milk ...,Amul,The scheme “will enable the creation of moder...
160542,Whose steps will destroy Assam and India?,Jinnah,"Instead of following Mahatma Gandhi, the Cong..."
187384,Where are the 9 medical institutes located?,Delhi,Indraprastha University will impart basic tra...
474336,"Along with Amma Ke Babu Ki Baby, what TV show ...",Vidya,"Actor Swadesh Mishra, who has been playing neg..."
194446,What is the name of the show that Gupta co-sta...,Netflix,Masaba Gupta co-starred with Satyadeep Misra ...


In [17]:
def model(df):
  precision = []  
  recall = []
  F1 = []
  n = df.shape[0]
  cnt = 1
  for i in range(n):
    if(cnt%100==0): 
        print(cnt) 
    question = Question(df.iloc[i][0])
    contexts = [Context(df.iloc[i][2])]
    candidates = bert_reader.predict(question, contexts)
    answer = get_best_answer(candidates, 0.5)
    predictions = [answer.text]
    references = [df.iloc[i][1]]
    results = scorer.score(predictions,references)
    precision.append(results[0].numpy()[0])
    recall.append(results[1].numpy()[0])
    F1.append(results[2].numpy()[0])
    cnt += 1

  f1 = sum(F1)/len(F1)
  pre = sum(precision)/len(precision)
  rec = sum(recall)/len(recall)

  return f1,pre,rec

In [18]:
res = model(df1)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700


6800
6900
7000
7100
7200
7300
7400
7500


7600


...

10800


10900
11000
11100
11200
11300
11400
11500
11600
11700
11800


11900
12000


12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100


14200
14300
14400


In [19]:
print("F1 = ",res[0])
print("Precision = ",res[1])
print("Recall = ",res[2])

F1 =  0.7370782464131707
Precision =  0.7097767681755506
Recall =  0.7689769904795741
